# Accessing FAME 4GL from Python
This example accesses FAME 4GL functionality from Python.

In [1]:
import os
import sys
from __future__ import print_function

import pandas as pd
from pyhli import *
import qoma_smuggler as qm

The Qoma utility function `open_hli()` opens the FAME environment and prints diagnostic information.

Below we use lower level FAME HLI functions. 
The `qomautils` package offers higher level composites of FAME HLI functions.  
The `qomautils` package function `open_hli()` calls `cfmini()` and `cfmver()` to initialize the FAME 
environment and to obtain FAME HLI version information.

In [2]:
if qm.open_hli()!=0:
    raise

Linux 4.14.33+ (#1 SMP Wed Jun 20 01:15:52 PDT 2018) x86_64
Python 2.7.5 (default, Apr 11 2018, 07:36:10) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-28)]
NumPy 1.14.5 Pandas 0.23.3 FAME HLI 11.63000 pyhli 0.0.6


FAME 4GL commands are easily passed to a FAME server using the FAME HLI function `cfmfame()`.  Here we:
*  set a date range to the prior quarter (`date thisday(q)-1`)
*  set the frequency to BUSINESS (`freq b`)
*  open a data base, instructing FAME to overwrite any existing database named `tmp.db` (`open<acc over>tmp`)
*  create a time series x, specifying `double` storage (`series x : precision by date`)
*  update the object's decription attribute (`desc(x) = "U[0,1]"`)
*  update the object's documentation (`docu(x) = "..."`)
*  use the FAME 4GL function `uniform()` to obtain samples from the uniform distribution indexed by date
*  direct FAME 4GL output to a temporary file, overwriting old file if needed (`output<acc over>tmp.txt`)
*  set report orientation to VERTICAL (time will appear vertically)
*  set report length to FULL (affects report pagination)
*  permit automatic time scale conversion (`conv on`)
*  request reports at `BUSINESS`, `WEEKLY(FRIDAY)`, `MONTHLY`, and `QUARTERLY` frequency.
*  FAME uses the object's OBSERVED attribute to properly reduce business data to lower frequency data.
*  instruct the FAME 4GL to close the output file (by directing output to terminal)
*  instruct FAME to close the database

Upon return to Python, we:
*  confirm the FAME HLI function `cfmfame()` returns code `HSUCC` (success)
*  display the temporary text file `tmp.txt`

In [3]:
cmd = ['\
  date thisday(q)-1; \
  freq b; \
  open<acc over>tmp; \
  series x : precision by date; \
  desc(x) = "U[0,1]"; \
  docu(x) = "Uniformly distributed U[0,1] time-series."; \
  set x = uniform(date); \
  output<acc over>tmp.txt; \
  whats x; \
  show vert; length full; conv on;\
  freq b; title text "Frequency "+@freq; repo x; \
  freq w(fri); title text "Frequency "+@freq; repo x; \
  freq m; title text "Frequency "+@freq; repo x; \
  freq q; title text "Frequency "+@freq; repo x; \
  output terminal; \
  close tmp\
  ']
cfmfame ([-1], cmd)
qm.print_file('tmp.txt')



                                   X

                                U[0,1]

Class:  SERIES                                     DB name:  TMP
Type:   PRECISION                                  Created:  31-Jul-18
Index:  DATE:BUSINESS                              Updated:  31-Jul-18

First Value at:  2-Apr-18                          Observed: SUMMED
Last Value at:  29-Jun-18                          Basis:    BUSINESS

Uniformly distributed U[0,1] time-series.



Frequency BUSINESS


              X
           ----

 2-Apr-18  0.65
 3-Apr-18  0.17
 4-Apr-18  0.03
 5-Apr-18  0.45
 6-Apr-18  0.20
 9-Apr-18  0.36
10-Apr-18  0.53
11-Apr-18  0.67
12-Apr-18  0.74
13-Apr-18  0.73
16-Apr-18  0.31
17-Apr-18  0.91
18-Apr-18  0.50
19-Apr-18  0.68
20-Apr-18  0.14
23-Apr-18  0.82
24-Apr-18  0.85
25-Apr-18  0.75
26-Apr-18  0.83
27-Apr-18  0.39
30-Apr-18  0.08
 1-May-18  0.46
 2-May-18  0.17
 3-May-18  0.18
 4-May-18  0.90
 7-May-18  0.44
 8-May-18  0.43
 9-May-18  0.87
10-May-18  0.45
11-May-18 

The `qomautils` function `read_fame()` reads FAME data objects into a nested Python dictionary.  
At the top level, each FAME object name is mapped to a dictionary with entries `data` and `fame`.  
*  As appropriate, the entry `data` maps to a single data value or to multiple data values in an array.  
    *  For FAME `SCALAR` objects, `data` maps to a value. 
    *  For FAME `SERIES` objects, `data` maps to an array of values.
*  The entry `fame` maps to FAME object meta data such as object class (`SCALAR` or `SERIES`), object data type, and index values for `data`.

We will use the `tmp` database constructed above.

In [4]:
dbname = "tmp"
famedata = qm.read_fame(dbname)
print("read_fame() returned {0} FAME objects from {1}.\n".format(len(famedata),dbname))

read_fame() returned 1 FAME objects from tmp.



Once FAME data objects are loaded to a Python dictionary, it is easy to access information.  First, a peek at the top level contents of the dictionary for the FAME data object `X` we constructed earlier in this notebook.

In [5]:
x = famedata.get('X')
print("data:\n{0}\n".format(x.get('data')))
print("fame:\n{0}\n".format(x.get('fame')))

data:
[0.6467646956443787, 0.1743934601545334, 0.03084705024957657, 0.4463737905025482, 0.2042222023010254, 0.36261826753616333, 0.5254757404327393, 0.6711335778236389, 0.7420555353164673, 0.7278105020523071, 0.31099629402160645, 0.9147082567214966, 0.5012004971504211, 0.676274299621582, 0.14207294583320618, 0.8199469447135925, 0.8483218550682068, 0.7456908822059631, 0.8264048099517822, 0.38549861311912537, 0.07541284710168839, 0.4637452960014343, 0.16738136112689972, 0.17849081754684448, 0.8950701355934143, 0.44424551725387573, 0.43421629071235657, 0.8734127283096313, 0.4474654495716095, 0.5516344308853149, 0.32012683153152466, 0.3715348243713379, 0.3859526515007019, 0.706091582775116, 0.2810337543487549, 0.33410021662712097, 0.22251564264297485, 0.820478081703186, 0.7750786542892456, 0.7468469738960266, 0.25690045952796936, 0.7259705066680908, 0.38640516996383667, 0.3116711378097534, 0.2566356062889099, 0.27455082535743713, 0.37556740641593933, 0.1613556146621704, 0.9038012623786926,

The meta data contained in the `fame` Python dictionary specifies things such as the date range of the `data`.

In [6]:
meta = x.get('fame')
rng = meta.get('range')
print("pandas range:\n{0}\n".format(qm.to_pandas_range(rng)))
print("description     {0}".format(meta.get('desc')))
print("documentation   {0}".format(meta.get('docu')))
print("fame class      {0}".format(qm.class_to_string(meta.get('class'))))
print("fame data type  {0}\n".format(qm.type_to_string(meta.get('type'))))

pandas range:
DatetimeIndex(['2018-04-02', '2018-04-03', '2018-04-04', '2018-04-05',
               '2018-04-06', '2018-04-09', '2018-04-10', '2018-04-11',
               '2018-04-12', '2018-04-13', '2018-04-16', '2018-04-17',
               '2018-04-18', '2018-04-19', '2018-04-20', '2018-04-23',
               '2018-04-24', '2018-04-25', '2018-04-26', '2018-04-27',
               '2018-04-30', '2018-05-01', '2018-05-02', '2018-05-03',
               '2018-05-04', '2018-05-07', '2018-05-08', '2018-05-09',
               '2018-05-10', '2018-05-11', '2018-05-14', '2018-05-15',
               '2018-05-16', '2018-05-17', '2018-05-18', '2018-05-21',
               '2018-05-22', '2018-05-23', '2018-05-24', '2018-05-25',
               '2018-05-28', '2018-05-29', '2018-05-30', '2018-05-31',
               '2018-06-01', '2018-06-04', '2018-06-05', '2018-06-06',
               '2018-06-07', '2018-06-08', '2018-06-11', '2018-06-12',
               '2018-06-13', '2018-06-14', '2018-06-15', '2018-

In [7]:
print(qm.meta_to_string(famedata,'X'))
qm.get(famedata,'X')

SERIES X : PRECISION BY DATE(BUSINESS) 2Apr2018 to 29Jun2018 -- U[0,1]
Uniformly distributed U[0,1] time-series.



2018-04-02    0.646765
2018-04-03    0.174393
2018-04-04    0.030847
2018-04-05    0.446374
2018-04-06    0.204222
2018-04-09    0.362618
2018-04-10    0.525476
2018-04-11    0.671134
2018-04-12    0.742056
2018-04-13    0.727811
2018-04-16    0.310996
2018-04-17    0.914708
2018-04-18    0.501200
2018-04-19    0.676274
2018-04-20    0.142073
2018-04-23    0.819947
2018-04-24    0.848322
2018-04-25    0.745691
2018-04-26    0.826405
2018-04-27    0.385499
2018-04-30    0.075413
2018-05-01    0.463745
2018-05-02    0.167381
2018-05-03    0.178491
2018-05-04    0.895070
2018-05-07    0.444246
2018-05-08    0.434216
2018-05-09    0.873413
2018-05-10    0.447465
2018-05-11    0.551634
                ...   
2018-05-21    0.334100
2018-05-22    0.222516
2018-05-23    0.820478
2018-05-24    0.775079
2018-05-25    0.746847
2018-05-28    0.256900
2018-05-29    0.725971
2018-05-30    0.386405
2018-05-31    0.311671
2018-06-01    0.256636
2018-06-04    0.274551
2018-06-05    0.375567
2018-06-06 

In [8]:
if qm.close_hli()!=0:
    raise

In [9]:
os.remove("tmp.txt")
os.remove("tmp.db")